In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib notebook
import numpy as np
import pandas as pd
from scipy import interpolate
import scipy as sp
import pickle as pkl
import copy

import xmeos
from xmeos import models
from xmeos import datamod
from xmeos import eoslib

from collections import OrderedDict
CONSTS = models.CONSTS

In [ ]:
def load_analysis(filename='data/analysis.pkl'):
    with open(filename, 'rb') as f:
        analysis = pkl.load(f) 
        
    return analysis
    
def save_analysis(analysis, filename='data/analysis.pkl'):
    with open(filename, 'wb') as f:
        pkl.dump(analysis, f) 
    

In [ ]:
def volume(P, T, eos_mod, Vinit=None, TOL=1e-3, bounds_error=True):


    def press_diff(V, P=P, T=T):
        return eos_mod.press(V, T) - P

    def press_diff_sqr(V, P=P, T=T):
        return (eos_mod.press(V, T) - P)**2

    def press_deriv(V, P=P, T=T):
        K = eos_mod.bulk_mod(V, T)
        dPdV = -K/V
        return dPdV

    
    if Vinit is None:
        V0, = eos_mod.get_param_values(param_names='V0')
        V_min0 = 0.8*V0
    else:
        V_min0 = Vinit
        
    V_min = sp.optimize.fmin(press_diff, V_min0, disp=False)

    #assert np.isscalar(Vinit), 'Vinit must be a scalar val.'
    # V = sp.optimize.fsolve(press_diff, Vinit, fprime=press_deriv)
    
    #V_min = sp.optimize.fmin(press_diff, Vinit, disp=False)

    if +press_diff(V_min) > +TOL:
        if bounds_error:
            raise ValueError(
                'The EOS is being sampled at an unphysical '
                'location! The target pressure is not accesible '
                'at this temperature.'
            )
        V = np.nan

    elif Vinit is None:
        output = sp.optimize.minimize(
            press_diff_sqr, [0.8*V_min], bounds=[(None, V_min)],
            options={'disp':False})
        V, = output.x

    else:
        #output = sp.optimize.minimize(
        #    press_diff_sqr, [Vinit], bounds=[(None, V_min)],
        #    options={'disp':False})
        #V, = output.x
        V = sp.optimize.fsolve(press_diff, Vinit, fprime=press_deriv)
    return V


In [ ]:
class HybridEos():
    def __init__(self, compress_eos, thermal_eos, PTS_fus0=None, 
                 fix_ref_adiabat=True):
        #, P_fus0=25, T_fus0=2900):
        
        self.fix_ref_adiabat = fix_ref_adiabat
        
        self.compress_eos = copy.deepcopy(compress_eos)
        self.thermal_eos = copy.deepcopy(thermal_eos)
        self.init_fus_refstate(PTS_fus0)
        
    def init_fus_refstate(self, PTS_fus0):
        try:
            P_fus0, T_fus0, S_fus0 = PTS_fus0
            
            assert P_fus0 is not None
            assert T_fus0 is not None
            assert S_fus0 is not None
        except:
            assert False, (
                'PTS_fus0 must provide fusion ref state. '
                'PTS_fus0 is a tuple (P_fus0, T_fus0, S_fus0).'
            )
        
        V_fus0 = self.volume(P_fus0, T_fus0)
        
        S0 = self.thermal_eos.get_param_values('S0')
        dS_adj = self.thermal_eos.entropy(V_fus0, T_fus0) - S_fus0
        
        S0 -= dS_adj
        self.thermal_eos.set_param_values(param_names='S0', param_values=S0)
        
        self.P_fus0 = P_fus0
        self.T_fus0 = T_fus0
        self.S_fus0 = S_fus0
        self.V_fus0 = V_fus0
        return
        
    def press(self, V, T):
        return self.compress_eos.press(V, T)
    
    def volume(self, P, T, Vinit=9):
        return volume(P, T, self.compress_eos, Vinit=Vinit)
    
    def bulk_mod(self, V, T):
        return self.compress_eos.bulk_mod(V, T)
    
    def thermal_exp(self, V, T):
        return self.compress_eos.thermal_exp(V, T)
    
    
    def heat_capacity(self, V, T):
        return self.thermal_eos.heat_capacity(V, T)
    
    def dPdT(self, V, T):
        dPdT = (self.compress_eos.press(V, T+.5)
                - self.compress_eos.press(V, T-.5))
        return dPdT
    
    def gamma(self, V, T):
        dPdT = self.dPdT(V, T)
        Cv = self.heat_capacity(V, T)
        gamma = V*dPdT/Cv/CONSTS['PV_ratio']
        return gamma
    
    def dTdV_ad(self, V, T):
        gamma = self.gamma(V, T)
        dTdV = -gamma*T/V
        return dTdV
    
    def sort_volumes(self, V, Vref):
        ind_sort = np.argsort(V)
        # if V.size==1:
        #     if V <= Vref:
        #         ind_contract = []
            
        try:
            mask_contract = V[ind_sort]<=Vref
        except:
            from IPython import embed; embed(); import ipdb as pdb; pdb.set_trace()
        
        ind_contract = ind_sort[mask_contract][::-1]
        ind_expand = ind_sort[~mask_contract]
        return ind_contract, ind_expand
    
    def entropy(self, V, T, TOL=1e-6):
        V_fus0 = self.V_fus0
        T_fus0 = self.T_fus0
        S_fus0 = self.S_fus0
        V = np.array(V)
        T = np.array(T)
        
        if not V.shape:
            V = np.array([V])
        if not T.shape:
            T = np.array([T])
        
        Vdev = 1-V/V_fus0
        Tdev = 1-T/T_fus0
        if V.size==1 and T.size==1 and np.all(np.abs(Vdev)<TOL) and np.all(np.abs(Tdev)<TOL):
            is_fus0_only = True
        else:
            is_fus0_only = False
            
            
        if is_fus0_only:
            S = S_fus0
        elif not self.fix_ref_adiabat:
            S = self.thermal_eos.entropy(V, T) 
        else:
            ind_contract, ind_expand = self.sort_volumes(V, V_fus0)
            
            
            if len(ind_contract)==0:
                has_contract = False
            else:
                has_contract = True
                
            if len(ind_expand)==0:
                has_expand = False
            else:
                has_expand = True
            
            if has_contract:
                if V[ind_contract][0] == V_fus0:
                    if(len(V[ind_contract])==1):
                        V_fus0_only = True
                    else:
                        V_fus0_only = False
                        
                    includes_V_fus0 = True
                    V_contract = V[ind_contract]
                else:
                    includes_V_fus0 = False
                    V_fus0_only = False
                    V_contract = np.hstack((V_fus0, V[ind_contract]))
                
            if has_expand:
                V_expand = np.hstack((V_fus0, V[ind_expand]))
                
            # print(V_contract)
            # print(V_expand)
            
            # deriv_fun = lambda V: self.dTdV_ad(V, )
            
            
            if has_contract and not V_fus0_only:
                output_contract = sp.integrate.solve_ivp(
                    self.dTdV_ad, V_contract[[0,-1]], np.array([self.T_fus0]), t_eval=V_contract)
                
                T_0S_contract = output_contract['y'][0]
            
            if has_expand:
                output_expand = sp.integrate.solve_ivp(
                    self.dTdV_ad, V_expand[[0,-1]], np.array([self.T_fus0]), t_eval=V_expand)
            
                T_0S_expand = output_expand['y'][0]
            
            
            T_0S = np.zeros(V.shape)
            
            if has_contract:
                if V_fus0_only:
                    T_0S[0] = self.T_fus0
                else:
                    if includes_V_fus0:
                        T_0S[ind_contract] = T_0S_contract
                    else:
                        T_0S[ind_contract] = T_0S_contract[1:]
            
            # T_0S[ind_contract] = T_0S_contract[1:]
            if has_expand:
                T_0S[ind_expand] = T_0S_expand[1:]
            
            # Now calc entropy diff using 
            dS_therm = (+self.thermal_eos.entropy(V, T)
                        -self.thermal_eos.entropy(V, T_0S))
            
            S = S_fus0 + dS_therm
            
            
        # 
        # dV = V[1]-V[0]
        # Vspan = [0, V[-1]]
        # 
        # output = sp.integrate.solve_ivp(self.dTdV_ad, Vspan, Tinit, 
        #                                 t_eval=V, vectorized=True)
        # 
        # 
        # dS = (liq_thermal_eos.entropy(V, T) 
        #       -liq_thermal_eos.entropy(V, Tad))
        
        # S = self.S_fus0 + dS
        return S
    def internal_energy():
        pass
    
    def adiabatic_path(self, Tinit, P_path):
        Pinit = P_path[0]
        Vinit = volume(Pinit, Tinit, self.compress_eos, Vinit=9)
        
        
        V = np.linspace(Vinit, Vinit*.4, 1001)
        Vspan = [Vinit, V[-1]]
        
        
        output = sp.integrate.solve_ivp(self.dTdV_ad, Vspan, np.array([Tinit]), 
                                        t_eval=V, vectorized=True)
        
        
        Tad = output['y'][0]
        # Pad = liq_compress_eos.press(V, Tad)
        
        return V, Tad

In [ ]:
V = np.linspace(.5,1.3, 101)
ind_rand = np.argsort(np.random.rand(V.size))
V = V[ind_rand]

def sort_volumes(V, V_fus0=1.0):
    ind_sort = np.argsort(V)
    mask_contract = V[ind_sort]<=V_fus0
    
    ind_contract = ind_sort[mask_contract][::-1]
    ind_expand = ind_sort[~mask_contract]
    return ind_contract, ind_expand

ind_contract, ind_expand = sort_volumes(V)

In [ ]:
def adjust_fusion_entropy(liq_eos, S_fus0, P_fus0, T_fus0, Vinit=9):
    Vliq_fus0 = volume(P_fus0, T_fus0, liq_eos, Vinit=Vinit)[0]
    # S0 = liq_eos_S11.get_param_values('S0')
    S0 = liq_eos.get_param_values('S0')
    dS_adj = liq_eos.entropy(Vliq_fus0, T_fus0) - S_fus0
    
    S0 = S0 - dS_adj
    liq_eos.set_param_values(param_names='S0', param_values=S0)
    print('S_fus0 diff = ', 
          liq_eos.entropy(Vliq_fus0, T_fus0)-S_fus0)

In [ ]:
def VT_deriv_fun_ad(P, VT, eos):
    V, T = VT[0], VT[1]
    KT = eos.bulk_mod(V, T)
    alpha = eos.thermal_exp(V, T)
    gamma = eos.gamma(V, T)
    
    KS = KT*(1+alpha*gamma*T)
    dVdP = -V/KS
    dTdP = gamma*T/KS
    
    VT_deriv = np.vstack((dVdP, dTdP))
    return VT_deriv

def integrate_adiabat(Pinit, Tinit, eos, Pmin=0.0, Pmax=150.0, N=101, Vinit_guess=9.0):
    deriv_fun = lambda P, VT, eos=eos: VT_deriv_fun_ad(P, VT, eos)
    
    Vinit = volume(Pinit, Tinit, eos, Vinit=Vinit_guess)
    # Vinit = eos.volume(Pinit, Tinit, Vinit_guess)
    
    P_path = np.linspace(Pinit, Pmax, N)
    output = sp.integrate.solve_ivp(deriv_fun, P_path[[0,-1]] , 
                                    np.array([Vinit, Tinit]), 
                                    t_eval=P_path, vectorized=True)
    V_ad_pos, T_ad_pos = output['y'][0], output['y'][1]
    
    
    P_path_neg = np.linspace(P_fus0, Pmin, N)
    output_neg = sp.integrate.solve_ivp(deriv_fun, P_path_neg[[0,-1]] , 
                                    np.array([Vinit, Tinit]), 
                                    t_eval=P_path_neg, vectorized=True)
    V_ad_neg, T_ad_neg = output_neg['y'][0], output_neg['y'][1]

    P_ad = np.hstack((P_path_neg[::-1][:-1], P_path))
    V_ad = np.hstack((V_ad_neg[::-1][:-1], V_ad_pos))
    T_ad = np.hstack((T_ad_neg[::-1][:-1], T_ad_pos))
    return P_ad, V_ad, T_ad
    

In [ ]:
# sp.integrate.solve_ivp?